In [ ]:
%pip install -U --quiet langchain-google-genai

In [69]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from google.generativeai.types.safety_types import HarmBlockThreshold, HarmCategory
import os

- pdf용

In [12]:
## 1. 로더(PDF시 PDF리더, TEXT시 단순 텍스트)
### 로더는 종류별로 각종 메타데이터(텍스트 이외의 정보 ex)페이지 번호 등)
loader = PyPDFLoader("한국어 4-24 토끼전.pdf")
pages = loader.load_and_split() ## 페이지별로 찢기

- 동화 종류

In [10]:
tale_dir = os.getcwd() + "/동화책"
tale_text_list = []

print(os.listdir(tale_dir))

for i in os.listdir(tale_dir) :    
    print(i)
    with open(tale_dir + "/" + i, 'r', encoding='UTF-8') as file:
        # # 파일의 모든 내용을 읽어서 문자열로 반환합니다.
        file_content = file.read()
        tale_text_list.append(file_content)

print("완료")

['개미와 베짱이.txt', '개와 고양이.txt', '견우와 직녀.txt', '단군 이야기.txt', '선녀와 나무꾼.txt', '소가 된 게으름뱅이.txt', '싸우기 좋아하는 아들들.txt', '엄마 게와 아기 게.txt', '여우와 신 포도.txt', '이슬 먹은 나귀.txt', '임금님 귀는 당나귀 귀.txt', '콩쥐팥쥐.txt', '토끼와 거북이.txt', '토끼전.txt', '해님과 바람.txt', '해와 달이 된 오누이.txt', '호랑이와 곶감.txt', '혹부리 영감.txt', '황금알을 낳는 거위.txt', '효녀 심청.txt', '흥부와 놀부.txt']
개미와 베짱이.txt
개와 고양이.txt
견우와 직녀.txt
단군 이야기.txt
선녀와 나무꾼.txt
소가 된 게으름뱅이.txt
싸우기 좋아하는 아들들.txt
엄마 게와 아기 게.txt
여우와 신 포도.txt
이슬 먹은 나귀.txt
임금님 귀는 당나귀 귀.txt
콩쥐팥쥐.txt
토끼와 거북이.txt
토끼전.txt
해님과 바람.txt
해와 달이 된 오누이.txt
호랑이와 곶감.txt
혹부리 영감.txt
황금알을 낳는 거위.txt
효녀 심청.txt
흥부와 놀부.txt
완료


In [11]:
## 텍스트 길이 => chunk의 필요는 없다
for i in tale_text_list:
    print(len(i))

420
953
658
701
842
854
612
434
415
451
575
910
604
722
563
585
593
852
571
710
778


In [8]:
# ## 2. 토크나이저(필요가 없다)
# text_splitter = RecursiveCharacterTextSplitter(separator = "\n\n", chunk_size=500, chunk_overlap=50, length_function = len) 
# texts = text_splitter.split_documents(pages)

In [33]:
# 3. 임베딩기 선정(Open ai 또는 Hugging face)
model_name = "jhgan/ko-sbert-nli" # 3. 임베딩기
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [34]:
## 4. VectorDB(Chroma)
DB_PATH = os.getcwd() + "/DB"
# docsearch = Chroma.from_texts(tale_text_list, hf, persist_directory=DB_PATH) 

## 4-1 DB통해 부르기
docsearch = Chroma(persist_directory=DB_PATH, embedding_function=hf)

In [35]:
## 5. 검색기
retriever = docsearch.as_retriever(
                                search_type="mmr",
                                search_kwargs={'k':1, 'fetch_k': 5}) ## k는 검색 유사도 문장 수
retriever.get_relevant_documents("제목 : 흥부") ## 이 후 chain.invoke question이 들어올 곳

[Document(page_content='제목 : 흥부와 놀부\n\n옛날 어느 마을에 형 놀부와 동생 흥부가 살았어요. \n흥부는 착한 사람이었어요. \n그런데 놀부는 욕심이 많고 나쁜 사람이었어요. \n어느 날 갑자기 아버지가 돌아가셨어요. \n놀부는 아버지의 집과 돈을 혼자 다 가졌어요. \n그리고 흥부네 가족을 내쫓았어요. \n흥부는 가난했지만 열심히 일했어요.\n\n어느 봄날, 제비 한 마리가\n흥부의 집에 날아왔어요. \n그런데 제비의 다리가 부러져 있었어요.\n흥부가 말했어요. \n“저런, 내가 고쳐줄게.” \n흥부는 제비의 다리를 고쳐 주었어요.\n\n다음 해 봄이 되자 제비가\n흥부의 집에 다시 돌아왔어요. \n그리고 흥부에게 박 씨 하나를 주었어요. \n흥부는 박 씨를 심었어요. \n가을이 되자 박이 주렁주렁 열렸어요. \n\n흥부의 가족들이 박을 잘랐어요. \n박 속에서 착한 도깨비들이 나와서\n큰 집과 돈을 주었어요.\n\n이 이야기를 들은 놀부는 샘이 났어요. \n그래서 제비를 잡아서 다리를 부러뜨렸어요. \n그리고 다시 고쳐 주었어요. \n다음 해 봄에 제비가 놀부에게도\n박 씨 하나를 주었어요.\n\n놀부는 박 씨 를 심었고 가을이 되자 박이 주렁주렁 열렸어요. \n놀부의 가족들이 박을 잘랐어요.\n\n그런데 놀부의 박 속에서는 나쁜 도깨비들이 나왔어요.\n\n도깨비들은 놀부의 집과 돈을 모두 빼앗아 갔어요.\n\n이 이야기를 들은 흥부는 놀부를 찾아 갔어요. \n흥부는 놀부의 가족을 자기 집으로 데리고 왔어요. \n놀부가 흥부에게 말했어요. \n“흥부야, 그동안 정말 미안했어.” \n착한 흥부는 놀부를 용서해 주었어요. \n그래서 흥부와 놀부는 사이좋게 살았어요.')]

In [111]:
## 프롬프트
template = """Make three quizs for {years} years old kids and return list like '[json, json, json]'.

json format:
"question" : "quiz",
"options" : ["option1","option2","option3"],
"answer" : 0

context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

harm_categories = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}


## 6. LLM 선정
os.environ['GOOGLE_API_KEY'] = ""
gemini = ChatGoogleGenerativeAI(model="gemini-pro", temperature = 1, settings=harm_categories)

## RAG
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents("제목 : " + x['topic']),
    "years" : lambda x:x['years']
}) | prompt | gemini

# 견우와 직녀/바보 온달 제외
print(chain.invoke({"topic": "흥부와 놀부", "years":"6"}).content)

[
  {
    "question": "형제 중에 착한 사람은 누구였나요?",
    "options": ["놀부", "흥부"],
    "answer": 1
  },
  {
    "question": "제비에게 무슨 일이 일어났었나요?",
    "options": ["날개가 부러졌다", "다리가 부러졌다", "코가 부러졌다"],
    "answer": 1
  },
  {
    "question": "박 속에서 나온 도깨비 중에 좋은 도깨비는 누구였나요?",
    "options": ["흥부의 도깨비", "놀부의 도깨비"],
    "answer": 0
  }
]
